# Intro to Building and Deploying an Agent with Agent Engine in Vertex AI
       


## Overview

### Agent Engine in Vertex AI

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) is a managed service that helps you to build and deploy an agent framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. Agent Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Agent Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.


### Objectives

In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.

You'll build and deploy an agent that uses the Gemini model, Python functions as tools, and LangChain for orchestration.

You will complete the following tasks:

- Install the Vertex AI SDK for Python
- Use the Vertex AI SDK to build components of a simple agent
- Test your agent locally before deploying
- Deploy and test your agent on Vertex AI
- Customize each layer of your agent (model, tools, orchestration)

## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Agent Engine and LangChain:

In [ ]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


In [ ]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
BUCKET_NAME = PROJECT_ID
STAGING_BUCKET = f"gs://{BUCKET_NAME}"
LOCATION = "us-central1"

import vertexai

vertexai.init(
    project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET
)

## Example: Build and deploy an agent

### Import libraries

In [ ]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import LangchainAgent

### Define model

As you construct your agent from the bottom up, the first component deals with which generative model you want to use in your agent.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-1.png" alt="Components of an agent in Agent Engine on Vertex AI" />

Here you'll use the Gemini 2.0 model:

In [ ]:
model = "gemini-2.0-flash"

### Define Python functions (tools)

The second component of your agent includes tools and functions, which enable the generative model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-2.png" alt="Components of an agent in Agent Engine on Vertex AI" />

In this example, you'll define a function called `get_exchange_rate` that uses the `requests` library to retrieve real-time currency exchange information from an API:

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

Test the function with sample inputs to ensure that it's working as expected:

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

### Define agent

The third component of your agent involves adding a reasoning layer, which helps your agent use the tools that you provided to help the end user achieve a higher-level goal.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-3.png" alt="Components of an agent in Agent Engine on Vertex AI" />

If you were to use Gemini and Function Calling on their own without a reasoning layer, you would need to handle the process of calling functions and APIs in your application code, and you would need to implement retries and additional logic to ensure that your function calling code is resilient to failures and malformed requests.

**Exercise:** Use the LangChain agent template provided in the Vertex AI SDK for Agent Engine to bring together the model, tools, and reasoning that you've built up so far. Use [this documentation](https://google.github.io/adk-docs/tools/third-party-tools/#1-using-langchain-tools) for reference.

In [ ]:
agent = LangchainAgent()  # TODO

Now we can test the model and agent behavior to ensure that it's working as expected before we deploy it:

### Test your agent locally

With all of the core components of your agent in place, you can send a prompt to your agent using `.query` to test that it's working as expected, including the intermediate steps that the agent performed between the input prompt and the generated summary output. In the default mode, the agent processes your input and returns the **entire agent output in a single response when complete**:

In [ ]:
agent.query(
    input="What's the exchange rate from US dollars to Indian currency latest ?"
)

In addition to the default query mode, the `.stream_query` method allows you to **see the agent's intermediate steps and final output from the chain**.

Instead of waiting for the agent to complete all sub-tasks, the agent sends back the response in **chunks as it's being generated**:

In [ ]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in agent.stream_query(
    input="What's the exchange rate from US dollars to Indian currency latest ?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])

This allows you to observe the agent's actions in real-time (such as function calls, and intermediate steps), which is helpful for debugging purposes or for providing real-time updates to the end user.

### Deploy your agent on Vertex AI

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-4.png" alt="Components of an agent in Agent Engine on Vertex AI" />

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [ ]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

**Exercise:** Now you're ready to deploy your agent to Agent Engine in Vertex AI. Use `agent_engines.create()` along with the below info to create the deployment:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.


In [ ]:
remote_agent = None  # TODO - use agent_engines.create()

Now you can send a prompt to your remote agent using `.query` to test that it's working as expected:

In [ ]:
remote_agent.query(input="What's the exchange rate from US dollars to Euro ?")

Or you can stream the results back from the remote agent interactively using `.stream_query`:

### Querying your deployed agent

You've now deployed your agent and can [interact with it in multiple ways](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), both within this notebook and from other applications or environments. The primary methods for accessing your deployed agent are via the Python client library or through REST API calls. Here's an overview of both methods:


In [ ]:
# List all agent engines
all_agent_engines = agent_engines.list()
print("All Agent Engines:")
for agent in all_agent_engines:
    print(f"- {agent.display_name} : {agent.resource_name}")

Use the resource name to load the agent in your other notebook or Python script, then query the remote agent as usual:

In [ ]:
RESOURCE_ID = "2598515412341620736"
remote_agent = agent_engines.get(RESOURCE_ID)

In [ ]:
remote_agent.query(input="What's the exchange rate from US dollars to Euro ?")

## Cleaning up

After you've finished, it's a good practice to clean up your cloud resources. You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [ ]:
remote_agent.delete()